In [1]:
import pandas as pd
import os

sectores = pd.read_csv('sectores.csv', sep=',') #mergear luego de los diccionarios
e2017 = pd.read_csv('e2017.csv', sep=',') #archivo a mergear con los diccionarios

In [2]:
files = os.listdir('/Users/JuanjoRodriguez/Dropbox/CEP-XXI/Trabajos en python/Fichas - AS/RAR/Inputs')
files = [x for x in files if ".csv" in x]
files

['10impoprueba.csv',
 'ARANCELES.csv',
 'ConversorH4aH3.csv',
 'ConversorH5aH3.csv',
 'ConversorHS1.CSV.csv',
 'ConversorHS2.CSV.csv',
 'ConversorHS3.CSV.csv',
 'DICCIONARIO.csv',
 'Dolar.csv',
 'empresasimpo2.csv',
 'EXPO.csv',
 'Fichas.csv',
 'Impo.csv',
 'IMPO_2017.csv',
 'IMPO_2017DATOSINVENTADOS.csv',
 'IMPORTACIONESPARAPESO.csv',
 'VBPFICHAS.csv']

In [3]:
from pathlib import Path
path = '/Users/JuanjoRodriguez/Dropbox/CEP-XXI/Trabajos en python/Fichas - AS/RAR/Inputs'
os.chdir(path)
diccionario = pd.read_csv('DICCIONARIO.csv', sep=';')
diccionario = diccionario.rename(columns={"HS":"posic_sim"}, inplace= False)
diccionario = diccionario.rename(columns={"CIIU":"ciiu"}, inplace= False)

In [4]:
expo2017 = pd.merge(e2017, diccionario, on="posic_sim", how='left')
expo2017 = expo2017.drop(['pdest', 'destino'], axis=1)
expo2017['posic_sim'] = expo2017['posic_sim'].apply(lambda x: str(x).zfill(6))
expo2017

anyo posic_sim         fob    ciiu
0      2017    100199  3083000.00   111.0
1      2017    100590    70720.00   111.0
2      2017    220421     7679.70  1552.0
3      2017    310510      141.00  2412.0
4      2017    854370     6790.00  3190.0
...     ...       ...         ...     ...
33816  2017    851680        4.21  2930.0
33817  2017    853650     5238.15  3120.0
33818  2017    854140     1149.20  3210.0
33819  2017    902519     1858.35  3312.0
33820  2017    100590   152790.20   111.0

[33821 rows x 4 columns]

In [5]:
missing = expo2017.loc[expo2017['ciiu'].isna(), :]
missing

anyo posic_sim       fob  ciiu
7554   2017    284800    800.00   NaN
13942  2017    847979  93346.75   NaN
27102  2017    030346      0.00   NaN
29370  2017    847979   6231.07   NaN
31346  2017    860500    414.04   NaN

In [6]:
expo2017 = expo2017.dropna()
expo2017

anyo posic_sim         fob    ciiu
0      2017    100199  3083000.00   111.0
1      2017    100590    70720.00   111.0
2      2017    220421     7679.70  1552.0
3      2017    310510      141.00  2412.0
4      2017    854370     6790.00  3190.0
...     ...       ...         ...     ...
33816  2017    851680        4.21  2930.0
33817  2017    853650     5238.15  3120.0
33818  2017    854140     1149.20  3210.0
33819  2017    902519     1858.35  3312.0
33820  2017    100590   152790.20   111.0

[33816 rows x 4 columns]

In [7]:
wd = os.getcwd()
dic_filenames = [x for x in os.listdir(wd) if '.CSV' in x and "HStoI3.CSV" !=x]
allHS = pd.DataFrame()

for f in sorted(dic_filenames): 
    print(f, "\n")
    df = pd.read_csv(f, sep=";")
    df.columns = [x.lower().lstrip().rstrip() for x in df.columns]
    df['hs'] = df['hs'].apply(lambda x: str(x).zfill(6))
    df['ciiu'] = df['ciiu'].apply(lambda x: str(x).zfill(4))
    df['HSVersion'] = f[:-8]
    allHS = allHS.append(df, ignore_index=True)
    print("File {} already appended\n".format(f))

ConversorHS1.CSV.csv 

File ConversorHS1.CSV.csv already appended

ConversorHS2.CSV.csv 

File ConversorHS2.CSV.csv already appended

ConversorHS3.CSV.csv 

File ConversorHS3.CSV.csv already appended



In [25]:
dic_filenames

['ConversorHS1.CSV.csv', 'ConversorHS2.CSV.csv', 'ConversorHS3.CSV.csv']

In [8]:
allHS=allHS[["hs","ciiu"]].drop_duplicates()
allHS = allHS.rename(columns={"hs":"posic_sim"}, inplace= False)
allHS

posic_sim  ciiu
0        070110  0111
1        070190  0111
2        071310  0111
3        071320  0111
4        071331  0111
...         ...   ...
15146    940389  3610
15210    950300  3694
15220    420600  3699
15285    961400  3699
15295    401199  4011

[5716 rows x 2 columns]

In [9]:
missing = pd.merge(missing, allHS, on='posic_sim', how='left')
missing

anyo posic_sim       fob  ciiu_x ciiu_y
0  2017    284800    800.00     NaN   2411
1  2017    847979  93346.75     NaN    NaN
2  2017    030346      0.00     NaN   1512
3  2017    847979   6231.07     NaN    NaN
4  2017    860500    414.04     NaN   3520

In [26]:
len(missing)

2

In [10]:
#crear ciiu2d
#expo2017['ciiu'] = expo2017['ciiu'].apply(lambda x: str(x).zfill(4))
#expo2017["ciiu2d"] = expo2017.ciiu.astype(str).str[:2].astype(int)

In [11]:
#cruzar con sectores
#expo2017 = pd.merge(expo2017, sectores, on='posic_sim', how='left')
#expo2017

In [12]:
#expo2017['ciiu2d'] = expo2017.ciiu.astype(str).str[:2].astype(int)

In [13]:
missing_asig = missing.drop('ciiu_x', axis=1)
missing_asig = missing_asig.dropna()
missing_asig = missing_asig.rename(columns={"ciiu_y":"ciiu"}, inplace= False)
missing_asig

anyo posic_sim     fob  ciiu
0  2017    284800  800.00  2411
2  2017    030346    0.00  1512
4  2017    860500  414.04  3520

In [14]:
missing = missing.loc[missing['ciiu_y'].isna(), :]
missing = missing.rename(columns={"ciiu_y":"ciiu"}, inplace= False)
missing

anyo posic_sim       fob  ciiu_x ciiu
1  2017    847979  93346.75     NaN  NaN
3  2017    847979   6231.07     NaN  NaN

In [15]:
missing2 = missing.drop('ciiu_x', axis=1)
missing2

anyo posic_sim       fob ciiu
1  2017    847979  93346.75  NaN
3  2017    847979   6231.07  NaN

In [16]:
H5toH3 = pd.read_csv('ConversorH5aH3.csv', sep=';')

H4toH3 = pd.read_csv('ConversorH4aH3.csv', sep=';')

H5toH3=H5toH3.rename(columns={"HS2017":"posic_sim"}, inplace= False)

H4toH3=H4toH3.rename(columns={"HS2012":"posic_sim"}, inplace= False)

In [17]:
missing2['posic_sim'] = missing2['posic_sim'].astype('int64')
missing_asig2 = pd.merge(missing2, H5toH3, on='posic_sim', how='left')
missing_asig2

anyo  posic_sim       fob ciiu  HS2007
0  2017     847979  93346.75  NaN  847989
1  2017     847979   6231.07  NaN  847989

In [18]:
missing_asig2 = missing_asig2.drop("HS2007", axis=1)
missing_asig2 = pd.merge(missing_asig2, H4toH3, on='posic_sim', how='left')
missing_asig2

anyo  posic_sim       fob ciiu  HS2007
0  2017     847979  93346.75  NaN  847989
1  2017     847979   6231.07  NaN  847989

In [19]:
pos_sin_ciiu = missing_asig2.drop('HS2007', axis=1)
pos_sin_ciiu

anyo  posic_sim       fob ciiu
0  2017     847979  93346.75  NaN
1  2017     847979   6231.07  NaN

In [20]:
expo2017 = expo2017.append(missing_asig)
expo2017

anyo posic_sim         fob    ciiu
0      2017    100199  3083000.00   111.0
1      2017    100590    70720.00   111.0
2      2017    220421     7679.70  1552.0
3      2017    310510      141.00  2412.0
4      2017    854370     6790.00  3190.0
...     ...       ...         ...     ...
33819  2017    902519     1858.35  3312.0
33820  2017    100590   152790.20   111.0
0      2017    284800      800.00    2411
2      2017    030346        0.00    1512
4      2017    860500      414.04    3520

[33819 rows x 4 columns]

In [21]:
expo2017["ciiu2d"] = expo2017.ciiu.astype(str).str[:2].astype(int)
expo2017sectores = pd.merge(expo2017, sectores, on='ciiu2d', how='left')
expo2017sectores

anyo posic_sim         fob    ciiu  ciiu2d sectores  \
0      2017    100199  3083000.00   111.0      11        C   
1      2017    100590    70720.00   111.0      11        C   
2      2017    220421     7679.70  1552.0      15      D15   
3      2017    310510      141.00  2412.0      24      D24   
4      2017    854370     6790.00  3190.0      31      D31   
...     ...       ...         ...     ...     ...      ...   
33814  2017    902519     1858.35  3312.0      33      D33   
33815  2017    100590   152790.20   111.0      11        C   
33816  2017    284800      800.00    2411      24      D24   
33817  2017    030346        0.00    1512      15      D15   
33818  2017    860500      414.04    3520      35      D35   

                           descripcion  
0      Explotación de minas y canteras  
1      Explotación de minas y canteras  
2                            Alimentos  
3                   Productos químicos  
4                  Aparatos eléctricos  
...                                ...  
33814             Instrumentos médicos  
33815  Explotación de minas y canteras  
33816               Productos químicos  
33817                        Alimentos  
33818       Otros equipo de transporte  

[33819 rows x 7 columns]

In [27]:
expo2017sectores = expo2017sectores.groupby(['sectores'])['fob'].sum()
expo2017sectores = expo2017sectores.to_frame()
expo2017sectores

fob
sectores              
C         1.369325e+10
D15       2.278591e+10
D16       2.981480e+07
D17       2.511318e+08
D18       5.512096e+07
D19       7.735942e+08
D20       1.255050e+08
D21       4.200507e+08
D22       3.717239e+07
D23       1.007986e+09
D24       5.514680e+09
D25       6.092031e+08
D26       1.602710e+08
D27       4.019645e+09
D28       3.344642e+08
D29       8.782224e+08
D30       1.788177e+07
D31       1.887342e+08
D32       5.615091e+07
D33       1.397921e+08
D34       6.121565e+09
D35       1.014180e+09
D36       8.183940e+07
E         6.446978e+05
G         1.210536e+06
K74       1.504451e+06
O         1.925525e+07

In [28]:
expo2017sectores.to_csv('expo2017_sectores.csv', sep=',')